In [1]:
# Install the required package
%pip install gimi9-geocoder

import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point
from gimi9_geocoder.client import GeocoderClient

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 초기화

geocoder = GeocoderClient(token="DEMO_TOKEN")

In [3]:
# 단순한 지오코딩

gdf = geocoder.geocode("서울특별시 강남구 테헤란로 123")

gdf

,address,z,h23_cd,hd_cd,bld_mgt_no,h1_nm,h23_nm,hd_nm,success,errmsg,pos_cd,addressCls,h1_cd,kostat_h1_cd,kostat_h2_cd,bn,rm,geometry
0,서울특별시 강남구 테헤란로 123,06133,11680,1168064000,1168010100106480024023791,서울,강남구,역삼1동,True,,도로명 주소,ROAD_ADDRESS,11,11,11230,1168010100106480024023791,테헤란로,POINT (127.03139 37.49955)


In [4]:
def create_map(gdf):
    # 1. Folium을 사용하여 배경 지도 생성
    # 지도의 중심 좌표를 GeoDataFrame의 평균 좌표로 설정
    # GeoDataFrame의 모든 geometry를 포함하는 경계 계산
    bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
    center_lat = (bounds[1] + bounds[3]) / 2
    center_lon = (bounds[0] + bounds[2]) / 2

    # Folium Map 초기화
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=12,
        tiles="CartoDB Positron"  # Alternative tile provider
    )

    # 모든 geometry가 화면에 나타나도록 지도 확대/축소 설정
    m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

    # 2. GeoDataFrame의 포인트 데이터를 지도에 추가
    # 각 포인트에 마커를 추가하고, 팝업에 이름을 표시
    for idx, row in gdf.iterrows():
        if row.geometry.geom_type == 'Point':
            folium.Marker(
                location=[row.geometry.y, row.geometry.x],
                popup=row['address'],
                icon=folium.Icon(color='red')
            ).add_to(m)
        else:
            if row.geometry.geom_type == 'Polygon' or row.geometry.geom_type == 'MultiPolygon':
                folium.GeoJson(
                    row.geometry,
                    # name=row['address'],
                    # popup=folium.Popup(row['address'], max_width=300)
                ).add_to(m)

    return m

map = create_map(gdf)

map


In [5]:

# list 지오코딩
addresses = [
        "서울특별시 강남구 테헤란로 123",
        "부산광역시 해운대구 해운대해변로 456",
        "대구광역시 중구 동성로 789",
    ]

gdf = geocoder.geocode(addresses)

gdf


,address,z,h23_cd,hd_cd,bld_mgt_no,h1_nm,h23_nm,hd_nm,success,errmsg,pos_cd,addressCls,h1_cd,kostat_h1_cd,kostat_h2_cd,bn,rm,geometry
0,서울특별시 강남구 테헤란로 123,06133,11680,1168064000,1168010100106480024023791,서울,강남구,역삼1동,True,,도로명 주소,ROAD_ADDRESS,11,11,11230,1168010100106480024023791,테헤란로,POINT (127.03139 37.49955)
1,부산광역시 해운대구 해운대해변로 456,NaN,26350,2635052500,NaN,부산,해운대,우제3동,True,도로명 대표주소: 해운대해변로,길이름 대표주소,ROAD_END_ADDRESS,26,21,21090,,해운대해변로,POINT (129.16694 35.165)
2,대구광역시 중구 동성로 789,NaN,27110,2711056500,NaN,대구,중,성내1동,True,도로명 대표주소: 동성로,길이름 대표주소,ROAD_END_ADDRESS,27,22,22010,,동성로,POINT (128.5965 35.8739)


In [6]:
map = create_map(gdf)

map

In [7]:
# df 지오코딩

data = {
    "name": ["주소1", "주소2", "주소3", "주소4", "주소5", "주소6", "주소7"],
    "address": [
            "서울특별시 중구 세종대로 110 서울특별시청",
            "부산광역시 연제구 중앙대로 1001(연산동) 부산광역시청",
            "대구광역시 중구 공평로 88 (동인동1가) 대구광역시청",
            "인천광역시 남동구 정각로29 (구월동 1138) 인천광역시청",
            "광주광역시 서구 내방로 111(치평동) 광주광역시청",
            "대전 서구 둔산로 100 대전광역시청",
            "한누리대로 2130 (보람동) 세종특별자치시청",
        ]
}
df = pd.DataFrame(data)

# 주소를 좌표로 변환 (지오코딩)
gdf = geocoder.geocode(df, address_col="address")

gdf

,name,address,z,h23_cd,hd_cd,bld_mgt_no,h1_nm,h23_nm,hd_nm,pos_cd,success,errmsg,addressCls,h1_cd,kostat_h1_cd,kostat_h2_cd,bn,rm,geometry
0,주소1,서울특별시 중구 세종대로 110 서울특별시청,04524,11140,1114055000,1114010300100310000019224,서울,중구,명동,도로명 주소,True,도로명 주소: 도로명 주소의 서울특별시청 (건물동 제외),ROAD_ADDRESS,11,11,11020,1114010300100310000019224,세종대로,POINT (126.97782 37.56631)
1,주소2,부산광역시 연제구 중앙대로 1001(연산동) 부산광역시청,47545,26470,2647069000,2647010200110000000022055,부산,연제구,연산제5동,도로명 주소,True,도로명 주소: 도로명 주소의 부산광역시청 (건물동 제외),ROAD_ADDRESS,26,21,21130,2647010200110000000022055,중앙대로,POINT (129.07509 35.17992)
2,주소3,대구광역시 중구 공평로 88 (동인동1가) 대구광역시청,41911,27110,2711051700,2711010100100010000009433,대구,중구,동인동,도로명 주소,True,도로명 주소: 도로명 주소의 대구광역시청 (건물동 제외),ROAD_ADDRESS,27,22,22010,2711010100100010000009433,공평로,POINT (128.60121 35.87155)
3,주소4,인천광역시 남동구 정각로29 (구월동 1138) 인천광역시청,21554,28200,2820052100,2820010100111380000000141,인천,남동구,구월3동,도로명 주소,True,도로명 주소: 도로명 주소의 인천광역시청 (건물동 제외),ROAD_ADDRESS,28,23,23050,2820010100111380000000141,정각로,POINT (126.7047 37.45625)
4,주소5,광주광역시 서구 내방로 111(치평동) 광주광역시청,61945,29140,2914074500,2914012000112000000003487,광주,서구,치평동,도로명 주소,True,도로명 주소: 도로명 주소의 광주광역시청 (건물동 제외),ROAD_ADDRESS,29,24,24020,2914012000112000000003487,내방로,POINT (126.85145 35.1601)
5,주소6,대전 서구 둔산로 100 대전광역시청,35242,30170,3017063000,3017011200114200000021946,대전,서구,둔산1동,도로명 주소,True,도로명 주소: 도로명 주소의 대전광역시청 (건물동 제외),ROAD_ADDRESS,30,25,25030,3017011200114200000021946,둔산로,POINT (127.38463 36.35038)
6,주소7,한누리대로 2130 (보람동) 세종특별자치시청,30151,36110,3611056000,3611010300106260005000001,세종,세종특별자치시,보람동,도로명 주소,True,도로명 주소: 도로명 주소의 세종특별자치시청 (건물동 제외),ROAD_ADDRESS,36,29,29010,3611010300106260005000001,한누리대로,POINT (127.28903 36.48005)


In [8]:
m = create_map(gdf)

m

In [9]:
# 좌표를 주소로 변환 (기본 리버스 지오코딩)
# GeoDataFrame 리턴. 지번 영역이 기본 geometry, 도로명 영역이 road_geometry
df_reversed = geocoder.reverse_geocode(x_col=127.031388, y_col=37.499545)

df_reversed


,ADR_MNG_NO,yyyymm_road,address_road,PNU,yyyymm_jibun,address_jibun,geometry,road_geometry
0,11680101312201000012300000,202404,서울특별시 강남구 테헤란로 123,1168010100106480024,202404,서울특별시 강남구 역삼동 648-24번지,"POLYGON ((127.03122 37.49965, 127.03135 37.499...","POLYGON ((127.031602 37.499508, 127.031281 37...."


In [10]:
# 리버스 지오코딩 (GeoDataFrame)
# GeoDataFrame 리턴. 기본 geometry 유지. 지번 영역이 jibun_geometry, 도로명 영역이 road_geometry

print("리버스 지오코딩 (DataFrame) 수행...")
df_reversed = geocoder.reverse_geocode(coordinates=gdf)

df_reversed


리버스 지오코딩 (DataFrame) 수행...


,name,address,z,h23_cd,hd_cd,bld_mgt_no,h1_nm,h23_nm,hd_nm,pos_cd,...,rm,geometry,jibun_geometry,road_geometry,ADR_MNG_NO,yyyymm_road,address_road,PNU,yyyymm_jibun,address_jibun
0,주소1,서울특별시 중구 세종대로 110 서울특별시청,04524,11140,1114055000,1114010300100310000019224,서울,중구,명동,도로명 주소,...,세종대로,POINT (126.97782 37.56631),"POLYGON ((126.977386 37.566972, 126.977738 37....","POLYGON ((126.978256 37.56619, 126.978217 37.5...",11140103200500100011000000,202404,서울특별시 중구 세종대로 110,1114010300100310000,202404,서울특별시 중구 태평로1가 31번지
1,주소2,부산광역시 연제구 중앙대로 1001(연산동) 부산광역시청,47545,26470,2647069000,2647010200110000000022055,부산,연제구,연산제5동,도로명 주소,...,중앙대로,POINT (129.07509 35.17992),"POLYGON ((129.072612 35.179628, 129.072634 35....","POLYGON ((129.07285 35.180558, 129.073268 35.1...",26470102200001000100100000,202404,부산광역시 연제구 중앙대로 1001,2647010200110000000,202404,부산광역시 연제구 연산동 1000번지
2,주소3,대구광역시 중구 공평로 88 (동인동1가) 대구광역시청,41911,27110,2711051700,2711010100100010000009433,대구,중구,동인동,도로명 주소,...,공평로,POINT (128.60121 35.87155),"POLYGON ((128.601246 35.871773, 128.601807 35....","POLYGON ((128.600847 35.871414, 128.600843 35....",27110101300701700008800000,202404,대구광역시 중구 공평로 88,2711010100100020001,202404,대구광역시 중구 동인동1가 2-1번지
3,주소4,인천광역시 남동구 정각로29 (구월동 1138) 인천광역시청,21554,28200,2820052100,2820010100111380000000141,인천,남동구,구월3동,도로명 주소,...,정각로,POINT (126.7047 37.45625),"POLYGON ((126.703948 37.455191, 126.704074 37....","POLYGON ((126.704455 37.455152, 126.703957 37....",28200101315307600002900000,202404,인천광역시 남동구 정각로 29,2820010100111380000,202404,인천광역시 남동구 구월동 1138번지
4,주소5,광주광역시 서구 내방로 111(치평동) 광주광역시청,61945,29140,2914074500,2914012000112000000003487,광주,서구,치평동,도로명 주소,...,내방로,POINT (126.85145 35.1601),"POLYGON ((126.850759 35.160642, 126.85087 35.1...","POLYGON ((126.848733 35.1604, 126.849902 35.16...",29140120316000400011100000,202404,광주광역시 서구 내방로 111,2914012000112000000,202404,광주광역시 서구 치평동 1200번지
5,주소6,대전 서구 둔산로 100 대전광역시청,35242,30170,3017063000,3017011200114200000021946,대전,서구,둔산1동,도로명 주소,...,둔산로,POINT (127.38463 36.35038),"POLYGON ((127.383108 36.34925, 127.383114 36.3...","POLYGON ((127.38357 36.350455, 127.383786 36.3...",30170112316601900010000000,202404,대전광역시 서구 둔산로 100,3017011200114200000,202404,대전광역시 서구 둔산동 1420번지
6,주소7,한누리대로 2130 (보람동) 세종특별자치시청,30151,36110,3611056000,3611010300106260005000001,세종,세종특별자치시,보람동,도로명 주소,...,한누리대로,POINT (127.28903 36.48005),"POLYGON ((127.28726 36.47884, 127.286937 36.47...","POLYGON ((127.288889 36.479722, 127.289654 36....",36110103200000200213000000,202404,세종특별자치시 한누리대로 2130,3611010300107180000,202404,세종특별자치시 보람동 718번지


In [11]:
# 리버스 지오코딩 (DataFrame)
# GeoDataFrame 리턴. 지번 영역이 기본 geometry, 도로명 영역이 road_geometry

dict_xy = {
    'x': {0: 126.97781828548713,
        1: 129.07508592851394,
        2: 128.60120913545674,
        3: 126.70470236162905,
        4: 126.85145206836815,
        5: 127.38462816763199,
        6: 127.2890337055837},
    'y': {0: 37.5663091836624,
        1: 35.17992240996954,
        2: 35.87154601594613,
        3: 37.45625459091278,
        4: 35.16009754366034,
        5: 36.35037987801865,
        6: 36.48005162632402}
}

df_xy = pd.DataFrame.from_dict(dict_xy)

df_reversed = geocoder.reverse_geocode(x_col="x", y_col="y", coordinates=df_xy)

df_reversed



,x,y,ADR_MNG_NO,yyyymm_road,address_road,PNU,yyyymm_jibun,address_jibun,geometry,road_geometry
0,126.977818,37.566309,11140103200500100011000000,202404,서울특별시 중구 세종대로 110,1114010300100310000,202404,서울특별시 중구 태평로1가 31번지,"POLYGON ((126.97739 37.56697, 126.97774 37.566...","POLYGON ((126.978256 37.56619, 126.978217 37.5..."
1,129.075086,35.179922,26470102200001000100100000,202404,부산광역시 연제구 중앙대로 1001,2647010200110000000,202404,부산광역시 연제구 연산동 1000번지,"POLYGON ((129.07261 35.17963, 129.07263 35.179...","POLYGON ((129.07285 35.180558, 129.073268 35.1..."
2,128.601209,35.871546,27110101300701700008800000,202404,대구광역시 중구 공평로 88,2711010100100020001,202404,대구광역시 중구 동인동1가 2-1번지,"POLYGON ((128.60125 35.87177, 128.60181 35.871...","POLYGON ((128.600847 35.871414, 128.600843 35...."
3,126.704702,37.456255,28200101315307600002900000,202404,인천광역시 남동구 정각로 29,2820010100111380000,202404,인천광역시 남동구 구월동 1138번지,"POLYGON ((126.70395 37.45519, 126.70407 37.456...","POLYGON ((126.704455 37.455152, 126.703957 37...."
4,126.851452,35.160098,29140120316000400011100000,202404,광주광역시 서구 내방로 111,2914012000112000000,202404,광주광역시 서구 치평동 1200번지,"POLYGON ((126.85076 35.16064, 126.85087 35.160...","POLYGON ((126.848733 35.1604, 126.849902 35.16..."
5,127.384628,36.350380,30170112316601900010000000,202404,대전광역시 서구 둔산로 100,3017011200114200000,202404,대전광역시 서구 둔산동 1420번지,"POLYGON ((127.38311 36.34925, 127.38311 36.350...","POLYGON ((127.38357 36.350455, 127.383786 36.3..."
6,127.289034,36.480052,36110103200000200213000000,202404,세종특별자치시 한누리대로 2130,3611010300107180000,202404,세종특별자치시 보람동 718번지,"POLYGON ((127.28726 36.47884, 127.28694 36.478...","POLYGON ((127.288889 36.479722, 127.289654 36...."


In [12]:
# 코드로 행정구역(행정동) 형상 검색
regions = geocoder.region(
    type="hd", code="1114055000,4111760000", yyyymm="202506"
)

regions

,success,type,code,name,yyyymm,geometry
0,True,hd,1114055000,명동,202506,"POLYGON ((126.98416 37.55306, 126.98298 37.555..."
1,True,hd,4111760000,광교1동,202506,"POLYGON ((127.06215 37.29629, 127.06502 37.291..."


In [13]:
# 코드로 행정구역(시군구) 형상 검색
regions = geocoder.region(
    type="h23", code="11140,41117", yyyymm="202506"
)

regions

,success,type,code,name,yyyymm,geometry
0,True,h23,11140,중구,202506,"POLYGON ((127.02339 37.57191, 127.02358 37.565..."
1,True,h23,41117,수원시 영통구,202506,"POLYGON ((127.07484 37.26588, 127.07732 37.265..."


In [14]:
map = create_map(regions)

map

In [15]:
# 코드로 행정구역(광역시도) 형상 검색
regions = geocoder.region(
    type="h1", code="11,41", yyyymm="202506"
)

regions

,success,type,code,name,yyyymm,geometry
0,True,h1,11,서울특별시,202506,"POLYGON ((126.82426 37.51447, 126.8231 37.5162..."
1,True,h1,41,경기도,202506,"MULTIPOLYGON (((126.54791 37.20575, 126.54521 ..."


In [16]:
# 행정동 이력
hd_history = geocoder.hd_history(x=127.075074, y=37.143834, yyyymm="202506")

hd_history

[{'hd_cd': '4137052300',
  'hd_nm': '대원1동',
  'hd_en': 'Daewon1(il)-dong',
  'from_yyyymm': '202312',
  'to_yyyymm': '202507'}]

In [17]:
# 토큰 통계
token_stats = geocoder.token_stats()

token_stats

{'total_requests': 389475,
 'last_month_requests': 7008,
 'monthly_quota': 100000000,
 'remaining_quota': 99992992,
 'daily_quota': 10000000,
 'daily_requests': 179,
 'remaining_daily_quota': 9999821,
 'last_request_date': '2025-08-22T01:03:14.602755+09:00'}